In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

from scipy.misc import imread, imresize
from sklearn.externals import joblib
import numpy as np 
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import convert, show_images, put_boxes, box_boundaries
from featuresourcer import FeatureSourcer
from binaryclassifier import BinaryClassifier
from slider import Slider
from heatmap import HeatMap 

In [ ]:
def hot_to_rgb(data):
  data_ = (data - np.min(data)) / (np.max(data) - np.min(data))
  my_cmap = plt.cm.get_cmap('hot')
  img = my_cmap(data_)
  rgb_img = np.dstack((img[:, :, 0], img[:, :, 1], img[:, :, 2]))
  return rgb_img

In [ ]:
svc = joblib.load('svc2.pkl')
scaler = joblib.load('scaler2.pkl')

sourcer_params = {
  'color_model': 'yuv',                # hls, hsv, yuv, ycrcb
  'bounding_box_size': 64,             #
  'number_of_orientations': 11,        # 6 - 12
  'pixels_per_cell': 16,               # 8, 16
  'cells_per_block': 2,                # 1, 2
  'do_transform_sqrt': True
}

cls = BinaryClassifier(svc, scaler)
src = FeatureSourcer(sourcer_params, imread("vehicles/KITTI_extracted/5364.png"))
slider = Slider(sourcer = src, classifier = cls, increment = 8)

In [ ]:
def verbose_pipeline(this_frame):

  for sz, pos in zip(ws, wp):
    bounding_boxes = slider.locate(frame = this_frame, window_size = sz, window_position = pos)
    heatmap.update(bounding_boxes)

  mp, _, _ = heatmap.get()
  labeled_img = heatmap.draw(this_frame)
  
  rgb_img = imresize(hot_to_rgb(mp), 0.25)
                     
  labeled_img[20:200, 20:340] = rgb_img
  
  return labeled_img

In [ ]:
ws = 80, 120, 150, 180
wp = 410, 390, 380, 380

temp_frame = imread("test1.jpg")
heatmap = HeatMap(frame = temp_frame, thresh = 25, memory = 40)

project_output = 'test_video_output.mp4'
clip1 = VideoFileClip("test_video.mp4");
white_clip = clip1.fl_image(verbose_pipeline) 
%time white_clip.write_videofile(project_output, audio = False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [ ]:
ws = 80, 120, 150
wp = 410, 390, 380

temp_frame = imread("test1.jpg")
heatmap = HeatMap(frame = temp_frame, thresh = 20, memory = 30)

project_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4");
white_clip = clip1.fl_image(verbose_pipeline) 
%time white_clip.write_videofile(project_output, audio = False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))